In [16]:
import pandas as pd
import numpy as np
import math
import random

In [17]:
df = pd.read_csv('../data/Asgmnt1_data.csv',header=None)

In [18]:
df.shape

(16000, 128)

In [19]:
normalized_rows = []
number_rows = df.shape[0]

for i in range(number_rows):
    normalized_row = []
    row_values = df.iloc[i, :]
    
    mean = row_values.mean()
    std = row_values.std(ddof=1)
    
    for val in row_values:
        normalized_val = (val - mean) / std
        normalized_row.append(normalized_val)
        
    normalized_rows.append(pd.Series(normalized_row, index=df.columns, name=df.index[i]))
    
normalized_data = pd.concat(normalized_rows, axis=1).T

In [20]:
normalized_data.shape

(16000, 128)

In [21]:
def calculate_euclidean_distance(row1, row2):
    sum_square = 0
    for (val1, val2) in zip(row1, row2):
        sum_square += (val1 - val2) ** 2
    
    return math.sqrt(sum_square)

In [22]:
def construct_euclidean_distance_matrix(dataframe):
    n = dataframe.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            distance = calculate_euclidean_distance(dataframe.iloc[i], dataframe.iloc[j])
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance
    
    return distance_matrix


In [ ]:
normalized_distance_matrix = construct_euclidean_distance_matrix(normalized_data)
np.savetxt("distances.csv", normalized_distance_matrix, delimiter=',', fmt='%.6f')

In [ ]:
normalized_distance_matrix.shape

In [ ]:
def haar_non_square_matrix(m, n):
    if n < m:
        raise ValueError("Number of columns must be greater than or equal to the number of rows")
    
    H = np.zeros((m, n))

    # First row is all ones
    H[0, :] = 1 / np.sqrt(n)
    
    # Build the rest of the matrix
    for level in range(1, m):
        step = n // (2 ** level)
        if step == 0:
            break  # Stop if the step becomes zero (more rows than meaningful steps)
        
        for i in range(0, n, 2 * step):
            H[level, i:i + step] = 1 / np.sqrt(step)
            H[level, i + step:i + 2 * step] = -1 / np.sqrt(step)
    
    return H

In [ ]:
h = haar_non_square_matrix(128, 16000).T

In [ ]:
wavelet_transformed_normalized_data = normalized_data * h

In [ ]:
wavelet_transformed_normalized_data.shape

In [ ]:
wavelet_transformed_distance_matrix = construct_euclidean_distance_matrix(wavelet_transformed_normalized_data.iloc[:, :4])

In [ ]:
def PCA(data_before_pca, n_principal_components):
    covariance_matrix = np.dot(data_before_pca.T, data_before_pca)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[indices]
    eigenvectors = eigenvectors[indices]
    components = eigenvectors[:, :n_principal_components]
    
    data_after_pca = np.dot(data_before_pca, components)
    
    return data_after_pca, eigenvectors[:n_principal_components] 

In [ ]:
pca_transformed_data, top_eigenvalues = PCA(normalized_data, 4)

In [ ]:
pca_transformed_data

In [ ]:
top_eigenvalues

In [ ]:
pca_transformed_distance_matrix = construct_euclidean_distance_matrix(pd.DataFrame(pca_transformed_data))

In [ ]:
np.savetxt('pca_transformed_distance_matrix.csv', pca_transformed_distance_matrix, delimiter=',', fmt='%.6f')
np.savetxt('pwavelet_transformed_distance_matrix.csv', wavelet_transformed_distance_matrix, delimiter=',', fmt='%.6f')

In [40]:
len(pca_transformed_distance_matrix)

TypeError: object of type 'NoneType' has no len()

In [7]:
def generate_random_pairs(matrix_size):
    row1 = -1, col1 = -2, row2 = -1, col2 = -2
    
    while (row1 == row2 and col1 == col2) or (row1 == col1 and row2 == col1):
        row1, col1 = random.randint(0, matrix_size-1), random.randint(0, matrix_size-1)
        row2, col2 = random.randint(0, matrix_size-1), random.randint(0, matrix_size-1)
    return (row1, col1), (row2, col2)

def compare_random_cells(matrix_list, n):
    matrix_size = len(matrices[0])
    for _ in range(n):
        pair1, pair2 = generate_random_pairs(matrix_size)
        
        for idx, matrix in enumerate(matrices):
            cell1 = matrix[pair1[0]][pair1[1]]
            cell2 = matrix[pair2[0]][pair2[1]]
            print(f"Matrix {idx+1} Comparison: Cell {pair1} = {cell1}, Cell {pair2} = {cell2}")

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2222891630.py, line 2)

In [15]:
matrices = [normalized_distance_matrix, wavelet_transformed_distance_matrix, pca_transformed_distance_matrix]

NameError: name 'normalized_distance_matrix' is not defined